# 环境准备和初始化

In [1]:
import ee
import geemap
import pandas as pd

geemap.set_proxy(port='7890')
ee.Authenticate()


True

In [2]:
# 初始化 Earth Engine
ee.Initialize()

# 哨兵2影像导出并保存获取时间

现在目前下载的方式是，以样地点为中心创建90 * 90m的矩形区域，下载下来后，以离样地坐标点的距离为标准，裁剪9 * 9的影像，后续以这个大小提取像元之间的纹理特征的时候，没有考虑到这个纹理特征是一个标量。

## 下载保存18-19的影像

### 筛选符合要求的影像并查看无效值数量

In [4]:
# 读取CSV文件
csv_file_path = r"/anaconda/envs/gee/project/树种识别/zhuzhou_tree_zb.csv"
df = pd.read_csv(csv_file_path)

# 选择前10个样地
df = df.head(10)

# 定义每年四个季节的时间段
years = ['2018']
seasons = {
    'spring': ['02-03', '05-04'],
    'summer': ['05-05', '08-06'],
    'autumn': ['08-07', '11-06'],
    'winter': ['11-07', '02-02']  # 跨年季节
}

# 创建生成90x90米范围的ROI函数
def create_roi(lat, lon):
    point = ee.Geometry.Point([lon, lat])
    roi = point.buffer(45).bounds()  # 缓冲区半径15米，生成90米*90米的矩形
    return roi

# 去云函数
def remove_cloud(image):
    qa = image.select('QA60')
    cloud_mask = qa.bitwiseAnd(1 << 10).eq(0)  # 云掩模
    cirrus_mask = qa.bitwiseAnd(1 << 11).eq(0)  # 高云掩模
    mask = cloud_mask.And(cirrus_mask)
    return image.updateMask(mask).divide(10000).select("B.*").copyProperties(image, ["system:time_start"])

# 无效值检测函数
def is_image_valid(image, region, threshold=0.3):
    # 统计有效像素和总像素（排除无效值和null值）
    valid_pixel_count = image.reduceRegion(
        reducer=ee.Reducer.count(),
        geometry=region,
        scale=10,
        maxPixels=1e13
    ).values().get(0)

    total_pixel_count = image.reduceRegion(
        reducer=ee.Reducer.count(),
        geometry=region,
        scale=10,
        maxPixels=1e13
    ).values().get(0)

    # 获取实际值并打印
    valid_pixel_count_info = valid_pixel_count.getInfo() if valid_pixel_count else None
    total_pixel_count_info = total_pixel_count.getInfo() if total_pixel_count else None

    # 打印信息进行调试，确认统计到的有效和总像素数
    print(f"Valid Pixel Count: {valid_pixel_count_info}")
    print(f"Total Pixel Count: {total_pixel_count_info}")

    # 如果无法获取有效像素或总像素数，或者总像素为0，则认为影像无效
    if valid_pixel_count_info is None or total_pixel_count_info is None or total_pixel_count_info == 0:
        print("Invalid image due to zero total pixels or None values.")
        return False

    # 计算无效像素比例
    try:
        invalid_ratio = 1 - (valid_pixel_count_info / total_pixel_count_info)
    except ZeroDivisionError:
        print("Error: Division by zero while calculating invalid ratio.")
        return False

    # 打印无效像素比例
    print(f"Invalid pixel ratio: {invalid_ratio * 100:.2f}%")

    # 如果无效像素比例大于阈值，则认为影像无效
    return invalid_ratio < threshold



# 输出结果
output = []

# 遍历每个样地
for index, row in df.iterrows():
    lat, lon = row['latitude'], row['longitude']
    sample_plot_id = row['sample plot id']
    roi = create_roi(lat, lon)

    # 遍历每年和每个季节，获取时间段内的符合条件的哨兵2影像
    for year in years:
        for season, date_range in seasons.items():
            # 如果是冬季，结束年份需要加1（跨年）
            start_date = f"{year}-{date_range[0]}"
            if season == 'winter':
                end_year = str(int(year) + 1)
                end_date = f"{end_year}-{date_range[1]}"
            else:
                end_date = f"{year}-{date_range[1]}"

            # 过滤影像并去除云量
            sentinel2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
                .filterDate(start_date, end_date) \
                .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
                .filterBounds(roi) \
                .map(remove_cloud) \
                .select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B11', 'B12'])

            # 获取影像的时间戳
            images = sentinel2.toList(sentinel2.size())

            # 存储该时间段内的影像个数和日期
            season_images = []
            for i in range(images.size().getInfo()):
                image = ee.Image(images.get(i))
                image_date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd').getInfo()

                # 检测影像是否有效
                if is_image_valid(image, roi):
                    # 获取并打印B4波段的影像值
                    b4_values = image.select('B4').reduceRegion(
                        reducer=ee.Reducer.mean(),
                        geometry=roi,
                        scale=10,
                        maxPixels=1e13
                    )
                    print(f"B4 Band Mean Value for {image_date}: {b4_values.get('B4').getInfo()}")

                    # 只保留每一天的第一张影像
                    if not any(img['date'] == image_date for img in season_images):
                        season_images.append({'date': image_date})

                # 如果影像数量超过10，停止添加
                if len(season_images) >= 20:
                    break

            # 打印结果
            print(f"样地ID: {sample_plot_id}, 年份: {year}, 季节: {season}")
            print(f"有效影像数量: {len(season_images)}")
            print(f"有效影像日期: {[img['date'] for img in season_images]}")
            print("-" * 50)

# 打印完成提示
print("影像统计打印完成。")


Valid Pixel Count: 81
Total Pixel Count: 81
Invalid pixel ratio: 0.00%
B4 Band Mean Value for 2018-03-09: 0.09870334713763701
样地ID: 43-422262-Y-02990, 年份: 2018, 季节: spring
有效影像数量: 1
有效影像日期: ['2018-03-09']
--------------------------------------------------


EEException: Collection.toList: The value of 'count' must be positive. Got: 0.

### 每个季节至多十张影像

#### 2018

In [4]:

import pandas as pd
import ee
import time

# 初始化地球引擎
ee.Initialize()

# # 延时2小时
# print("程序将在2小时后开始运行...")
# time.sleep(3 * 60 * 60)  # 延时2小时

# # 延时后开始执行代码
# print("2小时已经过去，开始执行任务")



csv_file_path = r"E:\Users\20747\Desktop\娄底、湘潭、长沙样地（少）.csv"
df = pd.read_csv(csv_file_path, encoding='gbk')

# 修改年份范围为2018-2024
years = [str(year) for year in range(2018, 2025)]  # 生成2018到2024的年份列表
seasons = {
    'spring': ['02-03', '05-04'],
    'summer': ['05-05', '08-06'],
    'autumn': ['08-07', '11-06'],
    'winter': ['11-07', '02-02']  # 跨年季节
}

# 创建生成90x90米范围的ROI函数
def create_roi(lat, lon):
    point = ee.Geometry.Point([lon, lat])
    roi = point.buffer(15).bounds()  # 缓冲区半径45米，生成90米*90米的矩形
    return roi

# 去云函数
def remove_cloud(image):
    qa = image.select('QA60')
    cloud_mask = qa.bitwiseAnd(1 << 10).eq(0)  # 云掩模
    cirrus_mask = qa.bitwiseAnd(1 << 11).eq(0)  # 高云掩模
    mask = cloud_mask.And(cirrus_mask)
    return image.updateMask(mask).divide(10000).select("B.*").copyProperties(image, ["system:time_start"])

# 导出影像函数
def export_image(image, sample_plot_id, year, season, date, region):
    export_name = f"{sample_plot_id}_{year}_{season}_{date}.tif"
    export_folder = os.path.join("Sentinel2", sample_plot_id, season)

    if not os.path.exists(export_folder):
        os.makedirs(export_folder)

    task = ee.batch.Export.image.toDrive(
        image=image,
        description=export_name,
        folder=export_folder,
        region=region,
        scale=10,
        crs='EPSG:4326',
        fileFormat='GeoTIFF'
    )
    task.start()

# 输出结果
output = []

# 用于追踪已启动的任务数量
started_tasks = 0
max_tasks = 2000  # 任务阈值

# 遍历每个样地
for index, row in df.iterrows():
    lat, lon = row['latitude'], row['longitude']
    sample_plot_id = row['sample plot id']
    roi = create_roi(lat, lon)

    for year in years:
        for season, date_range in seasons.items():
            start_date = f"{year}-{date_range[0]}"
            end_date = f"{year}-{date_range[1]}" if season != 'winter' else f"{int(year) + 1}-{date_range[1]}"

            sentinel2 = ee.ImageCollection('COPERNICUS/S2_HARMONIZED') \
                .filterDate(start_date, end_date) \
                .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
                .filterBounds(roi) \
                .map(remove_cloud) \
                .select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B11', 'B12'])

            # 检查影像集合是否为空
            image_count = sentinel2.size().getInfo()
            if image_count == 0:
                print(f"样地ID: {sample_plot_id}, 年份: {year}, 季节: {season} 没有符合条件的影像，跳过该季节。")
                continue

            season_images = []
            images = sentinel2.toList(image_count)

            for i in range(image_count):
                image = ee.Image(images.get(i))
                image_date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd').getInfo()

                # 只保留每一天的第一张影像
                if not any(img['date'] == image_date for img in season_images):
                    season_images.append({'date': image_date, 'image': image})

                if len(season_images) >= 10:
                    break

            if len(season_images) == 0:
                print(f"样地ID: {sample_plot_id}, 年份: {year}, 季节: {season} 没有有效影像，跳过该季节。")
                continue

            # 导出影像
            for img in season_images:
                export_image(img['image'], sample_plot_id, year, season, img['date'], roi)

            started_tasks += len(season_images)

            # 如果已启动的任务数量超过2000，等待2小时
            if started_tasks >= max_tasks:
                print(f"已启动 {started_tasks} 个任务，等待2小时...")
                time.sleep(2.5 * 60 * 60)  # 等待2小时
                started_tasks = 0  # 重置任务计数器

            print(f"样地ID: {sample_plot_id}, 年份: {year}, 季节: {season}")
            print(f"有效影像数量: {len(season_images)}")
            print("有效影像日期: ", [img['date'] for img in season_images])
            print("-" * 50)

print("影像导出任务已开始。")


样地ID: 43-412266-Y-01890, 年份: 2018, 季节: spring
有效影像数量: 6
有效影像日期:  ['2018-02-07', '2018-02-12', '2018-02-17', '2018-03-09', '2018-04-10', '2018-04-18']
--------------------------------------------------
样地ID: 43-412266-Y-01890, 年份: 2018, 季节: summer
有效影像数量: 7
有效影像日期:  ['2018-05-25', '2018-06-29', '2018-07-19', '2018-07-22', '2018-07-24', '2018-07-27', '2018-07-29']
--------------------------------------------------
样地ID: 43-412266-Y-01890, 年份: 2018, 季节: autumn
有效影像数量: 10
有效影像日期:  ['2018-08-13', '2018-08-21', '2018-09-05', '2018-09-15', '2018-09-30', '2018-10-02', '2018-10-05', '2018-10-07', '2018-10-12', '2018-10-27']
--------------------------------------------------
样地ID: 43-412266-Y-01890, 年份: 2018, 季节: winter
有效影像数量: 5
有效影像日期:  ['2018-11-24', '2018-11-26', '2018-11-29', '2018-12-16', '2019-01-23']
--------------------------------------------------
样地ID: 43-412266-Y-01890, 年份: 2019, 季节: spring
有效影像数量: 3
有效影像日期:  ['2019-03-11', '2019-04-08', '2019-05-03']
-------------------------------

In [9]:

import os
import pandas as pd
import ee
import time

# 初始化地球引擎
ee.Initialize()

# # 延时2小时
# print("程序将在2小时后开始运行...")
# time.sleep(3 * 60 * 60)  # 延时2小时

# # 延时后开始执行代码
# print("2小时已经过去，开始执行任务")



csv_file_path = r"E:\postgraduate\开题\huaihua_zuobiao_134_behind.csv"
df = pd.read_csv(csv_file_path, encoding='gbk')

# 定义每年四个季节的时间段
years = ['2021']
seasons = {
    'spring': ['02-03', '05-04'],
    'summer': ['05-05', '08-06'],
    'autumn': ['08-07', '11-06'],
    'winter': ['11-07', '02-02']  # 跨年季节
}

# 创建生成90x90米范围的ROI函数
def create_roi(lat, lon):
    point = ee.Geometry.Point([lon, lat])
    roi = point.buffer(15).bounds()  # 缓冲区半径45米，生成90米*90米的矩形
    return roi

# 去云函数
def remove_cloud(image):
    qa = image.select('QA60')
    cloud_mask = qa.bitwiseAnd(1 << 10).eq(0)  # 云掩模
    cirrus_mask = qa.bitwiseAnd(1 << 11).eq(0)  # 高云掩模
    mask = cloud_mask.And(cirrus_mask)
    return image.updateMask(mask).divide(10000).select("B.*").copyProperties(image, ["system:time_start"])

# 导出影像函数
def export_image(image, sample_plot_id, year, season, date, region):
    export_name = f"{sample_plot_id}_{year}_{season}_{date}.tif"
    export_folder = os.path.join("Sentinel2", sample_plot_id, season)

    if not os.path.exists(export_folder):
        os.makedirs(export_folder)

    task = ee.batch.Export.image.toDrive(
        image=image,
        description=export_name,
        folder=export_folder,
        region=region,
        scale=10,
        crs='EPSG:4326',
        fileFormat='GeoTIFF'
    )
    task.start()

# 输出结果
output = []

# 用于追踪已启动的任务数量
started_tasks = 0
max_tasks = 2000  # 任务阈值

# 遍历每个样地
for index, row in df.iterrows():
    lat, lon = row['latitude'], row['longitude']
    sample_plot_id = row['sample plot id']
    roi = create_roi(lat, lon)

    for year in years:
        for season, date_range in seasons.items():
            start_date = f"{year}-{date_range[0]}"
            end_date = f"{year}-{date_range[1]}" if season != 'winter' else f"{int(year) + 1}-{date_range[1]}"

            sentinel2 = ee.ImageCollection('COPERNICUS/S2_HARMONIZED') \
                .filterDate(start_date, end_date) \
                .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
                .filterBounds(roi) \
                .map(remove_cloud) \
                .select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B11', 'B12'])

            # 检查影像集合是否为空
            image_count = sentinel2.size().getInfo()
            if image_count == 0:
                print(f"样地ID: {sample_plot_id}, 年份: {year}, 季节: {season} 没有符合条件的影像，跳过该季节。")
                continue

            season_images = []
            images = sentinel2.toList(image_count)

            for i in range(image_count):
                image = ee.Image(images.get(i))
                image_date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd').getInfo()

                # 只保留每一天的第一张影像
                if not any(img['date'] == image_date for img in season_images):
                    season_images.append({'date': image_date, 'image': image})

                if len(season_images) >= 10:
                    break

            if len(season_images) == 0:
                print(f"样地ID: {sample_plot_id}, 年份: {year}, 季节: {season} 没有有效影像，跳过该季节。")
                continue

            # 导出影像
            for img in season_images:
                export_image(img['image'], sample_plot_id, year, season, img['date'], roi)

            started_tasks += len(season_images)

            # 如果已启动的任务数量超过2000，等待2小时
            if started_tasks >= max_tasks:
                print(f"已启动 {started_tasks} 个任务，等待2小时...")
                time.sleep(3 * 60 * 60)  # 等待2小时
                started_tasks = 0  # 重置任务计数器

            print(f"样地ID: {sample_plot_id}, 年份: {year}, 季节: {season}")
            print(f"有效影像数量: {len(season_images)}")
            print("有效影像日期: ", [img['date'] for img in season_images])
            print("-" * 50)

print("影像导出任务已开始。")


样地ID: 43-423231-N-00728, 年份: 2021, 季节: spring
有效影像数量: 10
有效影像日期:  ['2021-02-20', '2021-03-07', '2021-03-13', '2021-03-20', '2021-03-23', '2021-04-14', '2021-04-16', '2021-04-17', '2021-04-22', '2021-04-24']
--------------------------------------------------
样地ID: 43-423231-N-00728, 年份: 2021, 季节: summer
有效影像数量: 8
有效影像日期:  ['2021-05-29', '2021-06-06', '2021-06-16', '2021-06-18', '2021-06-21', '2021-07-16', '2021-07-20', '2021-08-04']
--------------------------------------------------
样地ID: 43-423231-N-00728, 年份: 2021, 季节: autumn
有效影像数量: 7
有效影像日期:  ['2021-08-07', '2021-08-19', '2021-08-27', '2021-09-06', '2021-09-19', '2021-09-26', '2021-10-06']
--------------------------------------------------
样地ID: 43-423231-N-00728, 年份: 2021, 季节: winter 没有符合条件的影像，跳过该季节。
样地ID: 43-412162-N-00699, 年份: 2021, 季节: spring
有效影像数量: 10
有效影像日期:  ['2021-02-20', '2021-03-07', '2021-03-13', '2021-03-20', '2021-03-23', '2021-04-14', '2021-04-16', '2021-04-17', '2021-04-22', '2021-04-24']
----------------------------

In [14]:
import threading

def task():
    print("4小时已经过去，开始执行任务")

# 延迟4小时执行
timer = threading.Timer(14400, task)
timer.start()

### 每个季节至多一张

#### 下载保存20-21年的影像

In [1]:
import ee
import pandas as pd
import csv

# 初始化地球引擎
ee.Initialize()

# 读取CSV文件
csv_file_path = r"/anaconda/envs/gee/project/树种识别/zhuzhou_tree_zb.csv"
df = pd.read_csv(csv_file_path)

# 定义每年四个季节的时间段
years = ['2020', '2021']
seasons = {
    'spring': ['02-03', '05-04'],
    'summer': ['05-05', '08-06'],
    'autumn': ['08-07', '11-06'],
    'winter': ['11-07', '02-02']  # 跨年季节
}

# 创建生成30x30米范围的ROI函数
def create_roi(lat, lon):
    point = ee.Geometry.Point([lon, lat])
    roi = point.buffer(45).bounds()  # 缓冲区半径15米，生成30米*30米的矩形
    return roi

# 去云函数
def remove_cloud(image):
    qa = image.select('QA60')
    cloud_mask = qa.bitwiseAnd(1 << 10).eq(0)  # 云掩模
    cirrus_mask = qa.bitwiseAnd(1 << 11).eq(0)  # 高云掩模
    mask = cloud_mask.And(cirrus_mask)
    return image.updateMask(mask).divide(10000).select("B.*").copyProperties(image, ["system:time_start"])

# 打开CSV文件以记录影像时间和样地ID
output_csv_path = r"/anaconda/envs/gee/project/树种识别/S2_20-21_dates.csv"

with open(output_csv_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['sample_plot_id', 'year', 'season', 'image_date'])

    # 导出影像函数
    def export_image(image, folder_name, file_name, region):
        task = ee.batch.Export.image.toDrive(
            image=image,
            description=file_name,
            folder=f"{folder_name}/{file_name.split('_')[0]}",  # 创建对应样地的文件夹
            scale=10,
            region=region,
            crs='EPSG:4326',
            skipEmptyTiles=True,
            maxPixels=1e13
        )
        task.start()

    # 遍历每个样地并导出对应的影像
    for index, row in df.iterrows():
        lat, lon = row['latitude'], row['longitude']
        sample_plot_id = row['sample plot id']
        roi = create_roi(lat, lon)

        # 遍历每年和每个季节，获取时间段内的符合条件的哨兵2影像
        for year in years:
            for season, date_range in seasons.items():
                # 如果是冬季，结束年份需要加1（跨年）
                start_date = f"{year}-{date_range[0]}"
                if season == 'winter':
                    end_year = str(int(year) + 1)
                    end_date = f"{end_year}-{date_range[1]}"
                else:
                    end_date = f"{year}-{date_range[1]}"

                # 过滤影像并去除云量
                sentinel2 = ee.ImageCollection('COPERNICUS/S2_HARMONIZED')\
                    .filterDate(start_date, end_date)\
                    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 50))\
                    .filterBounds(roi)\
                    .map(remove_cloud)\
                    .select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B11', 'B12'])

                # 选择时间段内的第一幅符合条件的影像
                sentinel2_image = sentinel2.first().clip(roi)

                # 提取影像的获取时间
                image_date = ee.Date(sentinel2_image.get('system:time_start')).format('YYYY-MM-dd').getInfo()

                # 检查影像是否存在
                count = sentinel2_image.reduceRegion(
                    reducer=ee.Reducer.count(),
                    geometry=roi,
                    scale=10,
                    maxPixels=1e13
                ).values().get(0).getInfo()

                if count > 0:
                    sentinel2_folder_name = 'S2_20-21'
                    sentinel2_file_name = f"{sample_plot_id}_S2_{season}_{year}"
                    # 记录样地ID、年份、季节和影像时间到CSV文件
                    writer.writerow([sample_plot_id, year, season, image_date])
                    # 导出哨兵2影像
                    export_image(sentinel2_image, sentinel2_folder_name, sentinel2_file_name, roi)

print("哨兵2影像获取时间及样地ID已保存到:", output_csv_path)


哨兵2影像获取时间及样地ID已保存到: /anaconda/envs/gee/project/树种识别/S2_20-21_dates.csv


#### 下载保存22-23年的影像

In [2]:
import ee
import pandas as pd
import csv

# 初始化地球引擎
ee.Initialize()

# 读取CSV文件
csv_file_path = r"/anaconda/envs/gee/project/树种识别/zhuzhou_tree_zb.csv"
df = pd.read_csv(csv_file_path)

# 定义每年四个季节的时间段
years = ['2022', '2023']
seasons = {
    'spring': ['02-03', '05-04'],
    'summer': ['05-05', '08-06'],
    'autumn': ['08-07', '11-06'],
    'winter': ['11-07', '02-02']  # 跨年季节
}

# 创建生成30x30米范围的ROI函数
def create_roi(lat, lon):
    point = ee.Geometry.Point([lon, lat])
    roi = point.buffer(45).bounds()  # 缓冲区半径15米，生成30米*30米的矩形
    return roi

# 去云函数
def remove_cloud(image):
    qa = image.select('QA60')
    cloud_mask = qa.bitwiseAnd(1 << 10).eq(0)  # 云掩模
    cirrus_mask = qa.bitwiseAnd(1 << 11).eq(0)  # 高云掩模
    mask = cloud_mask.And(cirrus_mask)
    return image.updateMask(mask).divide(10000).select("B.*").copyProperties(image, ["system:time_start"])

# 打开CSV文件以记录影像时间和样地ID
output_csv_path = r"/anaconda/envs/gee/project/树种识别/S2_22-23_dates.csv"

with open(output_csv_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['sample_plot_id', 'year', 'season', 'image_date'])

    # 导出影像函数
    def export_image(image, folder_name, file_name, region):
        task = ee.batch.Export.image.toDrive(
            image=image,
            description=file_name,
            folder=f"{folder_name}/{file_name.split('_')[0]}",  # 创建对应样地的文件夹
            scale=10,
            region=region,
            crs='EPSG:4326',
            skipEmptyTiles=True,
            maxPixels=1e13
        )
        task.start()

    # 遍历每个样地并导出对应的影像
    for index, row in df.iterrows():
        lat, lon = row['latitude'], row['longitude']
        sample_plot_id = row['sample plot id']
        roi = create_roi(lat, lon)

        # 遍历每年和每个季节，获取时间段内的符合条件的哨兵2影像
        for year in years:
            for season, date_range in seasons.items():
                # 如果是冬季，结束年份需要加1（跨年）
                start_date = f"{year}-{date_range[0]}"
                if season == 'winter':
                    end_year = str(int(year) + 1)
                    end_date = f"{end_year}-{date_range[1]}"
                else:
                    end_date = f"{year}-{date_range[1]}"

                # 过滤影像并去除云量
                sentinel2 = ee.ImageCollection('COPERNICUS/S2_HARMONIZED')\
                    .filterDate(start_date, end_date)\
                    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 50))\
                    .filterBounds(roi)\
                    .map(remove_cloud)\
                    .select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B11', 'B12'])

                # 选择时间段内的第一幅符合条件的影像
                sentinel2_image = sentinel2.first().clip(roi)

                # 提取影像的获取时间
                image_date = ee.Date(sentinel2_image.get('system:time_start')).format('YYYY-MM-dd').getInfo()

                # 检查影像是否存在
                count = sentinel2_image.reduceRegion(
                    reducer=ee.Reducer.count(),
                    geometry=roi,
                    scale=10,
                    maxPixels=1e13
                ).values().get(0).getInfo()

                if count > 0:
                    sentinel2_folder_name = 'S2_22-23'
                    sentinel2_file_name = f"{sample_plot_id}_S2_{season}_{year}"
                    # 记录样地ID、年份、季节和影像时间到CSV文件
                    writer.writerow([sample_plot_id, year, season, image_date])
                    # 导出哨兵2影像
                    export_image(sentinel2_image, sentinel2_folder_name, sentinel2_file_name, roi)

print("哨兵2影像获取时间及样地ID已保存到:", output_csv_path)


哨兵2影像获取时间及样地ID已保存到: /anaconda/envs/gee/project/树种识别/S2_22-23_dates.csv


### 样地点的查看

In [16]:
import ee
import geemap
import pandas as pd

# 初始化地球引擎
ee.Initialize()

# 创建地图
Map = geemap.Map()

# 读取CSV文件
csv_file_path = r"/anaconda/envs/gee/project/树种识别/zhuzhou_tree.csv"
df = pd.read_csv(csv_file_path)

# 定义时间段（四个季节）
seasons = {
    'spring': ['2018-02-03', '2018-05-04'],
    'summer': ['2018-05-05', '2018-08-06'],
    'autumn': ['2018-08-07', '2018-11-06'],
    'winter': ['2018-11-07', '2019-02-02']  # 冬季跨年
}

# 创建生成30x30米范围的ROI函数
def create_roi(lat, lon):
    point = ee.Geometry.Point([lon, lat])
    roi = point.buffer(15).bounds()  # 缓冲区半径15米，生成30米*30米的矩形
    return roi

# 去云函数
def remove_cloud(image):
    qa = image.select('QA60')
    cloud_mask = qa.bitwiseAnd(1 << 10).eq(0)  # 云掩模
    cirrus_mask = qa.bitwiseAnd(1 << 11).eq(0)  # 高云掩模
    mask = cloud_mask.And(cirrus_mask)
    return image.updateMask(mask).copyProperties(image, ["system:time_start"])

# 检查影像是否在ROI区域内有云
def is_image_cloudy(image, roi):
    qa = image.select('QA60')
    cloud_mask = qa.bitwiseAnd(1 << 10).neq(0)  # 云掩模：QA60波段第10位表示云
    cloudy_pixels = image.updateMask(cloud_mask)  # 用掩模标记云
    cloudy_pixel_count = cloudy_pixels.reduceRegion(
        reducer=ee.Reducer.count(),
        geometry=roi,
        scale=10,
        maxPixels=1e13
    ).values().get(0)

    # 检查云像素数量，如果大于0，说明有云
    return cloudy_pixel_count.getInfo() > 0

# 遍历前5个样地，检测每个季节的影像
for index, row in df.iterrows():
    if index >= 200:  # 只处理前5个样地
        break

    lat, lon = row['latitude'], row['longitude']
    sample_plot_id = row['sample plot id']
    roi = create_roi(lat, lon)

    for season, date_range in seasons.items():
        # 获取2018年内每个季节的影像，云量过滤50%
        sentinel2 = ee.ImageCollection('COPERNICUS/S2_HARMONIZED')\
            .filterDate(date_range[0], date_range[1])\
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 50))\
            .filterBounds(roi)\
            .map(remove_cloud)\
            .select(['B2', 'B3', 'B4', 'QA60'])

        # 选择时间段内的第一张影像
        sentinel2_image = sentinel2.first().clip(roi)

        # 检查影像是否存在
        count = sentinel2_image.reduceRegion(
            reducer=ee.Reducer.count(),
            geometry=roi,
            scale=10,
            maxPixels=1e13
        ).values().get(0).getInfo()

        if count > 0:
            # 检查影像是否有云
            if is_image_cloudy(sentinel2_image, roi):
                print(f"Sample Plot ID: {sample_plot_id}, Season: {season}, Image has clouds.")
                # 将有云的影像添加到地图上
                Map.addLayer(sentinel2_image, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}, f"{sample_plot_id}_{season}_cloudy")

# 显示地图
Map.centerObject(roi, 10)  # 将地图中心对准第一个样地
Map.addLayerControl()  # 添加图层控制
Map


Map(center=[27.264059696354696, 113.74659163912786], controls=(WidgetControl(options=['position', 'transparent…